In [2]:
import numpy as np
import pandas as pd
from scipy.special import expit
import tensorflow as tf
from sklearn.model_selection import train_test_split
from data.Twins import Twins
from api.models.ganite import GANITE
import itertools
import logging
# switch to logging.DEBUG to debug model
logging.basicConfig(level=logging.INFO)

In [3]:
def PEHE(y, hat_y):
    e_PEHE = tf.reduce_mean(
        tf.squared_difference((y[:, 1] - y[:, 0]),
                              (hat_y[:, 1] - hat_y[:, 0])))
    return e_PEHE


def ATE(y, hat_y):
    e_PEHE = tf.abs(
        tf.reduce_mean(y[:, 1] - y[:, 0]) -
        tf.reduce_mean(hat_y[:, 1] - hat_y[:, 0]))
    return e_PEHE

In [7]:
twins = Twins()
num_patients, num_features = twins.X.shape
opt_y = twins.one_year_mortality(twins.Y)
T = treatment_assignment(twins.X)
Y = observable_outcomes(opt_y, T)

In [ ]:
# run each algorithm 100 times with different training/validation/testing splits

In [8]:
# Split data into 56/24/20 train, validation, test
train_X, test_X, train_T, test_T, train_Y, test_Y, train_OptY, test_OptY = train_test_split(
    twins.X, T, Y, opt_y, test_size=0.2)
train_X, validate_X, train_T, validate_T, train_Y, validate_Y, train_OptY, validate_OptY = train_test_split(
    train_X, train_T, train_Y, train_OptY, test_size=0.3)

dim_outcome = test_OptY.shape[1]

In [9]:
# hyperparameters
mini_batch_size = {32, 64, 128, 256}
alpha = {0, 0.1, 0.5, 1, 2, 5, 10}
h_dim = {num_features, np.ceil(num_features/2.), np.ceil(num_features/3.), np.ceil(num_features/4.), np.ceil(num_features/5.)}

In [ ]:
# Maybe could using K fold CV search here but with do 56/24/20 split to match paper methodology

In [10]:
num_iterations = 1000
num_kk = 10
results = {}
for _alpha, _mini_batch_size, _h_dim in itertools.product(alpha, mini_batch_size, h_dim):
    model = GANITE(_alpha, _mini_batch_size, num_iterations, num_kk, int(_h_dim))
    model.fit(train_X, train_T, train_Y, dim_outcome)
    hat_Y = model.predict(test_X)
    pehe = model.test(test_X, test_OptY, metric=PEHE)
    results[(_alpha, _mini_batch_size, _h_dim)] = pehe

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



100%|██████████| 1000/1000 [00:01<00:00, 672.23it/s]


100%|██████████| 1000/1000 [00:01<00:00, 609.89it/s]


In [36]:
for k in results:
    if results[k] == min(results.values()):
        print(k)

(0.1, 256, 10.0)


In [31]:
import json
with open('results.json', 'w') as f:
    f.write(json.dumps({str(k):str(v) for k,v in results.items()}))

In [33]:
min(results.values())

0.08407481

In [ ]:
test_OptY

In [38]:
model = GANITE(0.1, 256, num_iterations, num_kk, int(10))
model.fit(train_X, train_T, train_Y, dim_outcome)
hat_Y = model.predict(test_X)
pehe = model.test(test_X, test_OptY, metric=PEHE)


100%|██████████| 1000/1000 [00:01<00:00, 641.89it/s]


In [ ]:
# CMGP

In [ ]:
cmgp = CMGP(dim=30)
cmgp.fit(Train_X, Train_Y, Train_T)

In [51]:
import GPy
dim=30
X, Y, W = train_X, train_Y, train_T
df = pd.DataFrame(X)
df['Y'] = Y
df['W'] = W
feature_cols = [col for col in df.columns if col not in ['Y', 'W']]
X0 = df.loc[df['W'] == 0, feature_cols]
y0 = df.loc[df['W'] == 0, ['Y']]
X1 = df.loc[df['W'] == 1, feature_cols]
y1 = df.loc[df['W'] == 1, ['Y']]

K0 = GPy.kern.RBF(dim, ARD=True)
K1 = GPy.kern.RBF(dim, ARD=True)

In [ ]:
kernel = GPy.util.multioutput.LCM(input_dim=dim,
                                          num_outputs=2,
                                          kernels_list=[K0, K1])
model = GPy.models.GPCoregionalizedRegression(X_list=[X0, X1],
                                                           Y_list=[y0, y1],
                                                           kernel=kernel)

INFO:GP:initializing Y
INFO:GP:initializing inference method
INFO:GP:adding kernel and likelihood as parameters
